In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import fileinput, glob, os
import datetime as dt
import missingno as msno
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [8]:
stock_list = pd.read_csv('../files/stock_list.csv', index_col=0, parse_dates=[1])
stock_list_incl = stock_list[stock_list.Universe0 == True]
stock_list_incl.TradeDate = pd.to_datetime(stock_list_incl.TradeDate, format='%Y%m%d')
stock_list_incl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1301 to 9997
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EffectiveDate           2000 non-null   datetime64[ns]
 1   Name                    2000 non-null   object        
 2   Section/Products        2000 non-null   object        
 3   NewMarketSegment        2000 non-null   object        
 4   33SectorCode            2000 non-null   object        
 5   33SectorName            2000 non-null   object        
 6   17SectorCode            2000 non-null   object        
 7   17SectorName            2000 non-null   object        
 8   NewIndexSeriesSizeCode  2000 non-null   object        
 9   NewIndexSeriesSize      2000 non-null   object        
 10  TradeDate               2000 non-null   datetime64[ns]
 11  Close                   2000 non-null   float64       
 12  IssuedShares            2000 non-null   float

/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/2737920863.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_list_incl.TradeDate = pd.to_datetime(stock_list_incl.TradeDate, format='%Y%m%d')


In [2]:
#Import from stock_prices.csv and parse columns with dates:
stock_prices = pd.read_csv('../files/train_files/stock_prices.csv', index_col=0, parse_dates=[1])
stock_prices

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
RowId,,,,,,,,,,,
20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...
20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302


In [3]:
#inputting test dates
test = pd.read_csv('../files/example_test_files/stock_prices.csv', index_col=0, parse_dates=[1])
test['Target'] = 0
test[test['Date'] == '2021-12-06']

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
RowId,,,,,,,,,,,
20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,0
20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,0
20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False,0
20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False,0
20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False,0
...,...,...,...,...,...,...,...,...,...,...,...
20211206_9990,2021-12-06,9990,528.0,531.0,516.0,517.0,65300,1.0,NaN,False,0
20211206_9991,2021-12-06,9991,796.0,800.0,785.0,785.0,29100,1.0,NaN,False,0
20211206_9993,2021-12-06,9993,1645.0,1653.0,1627.0,1627.0,6200,1.0,NaN,False,0


In [4]:
stock_prices = pd.concat([stock_prices, test])
stock_prices[stock_prices['Date'] == '2021-12-06'].shape

(2000, 11)

In [5]:
stock_prices.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
RowId,,,,,,,,,,,
20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [6]:
#Logging the prices
stock_prices['Close'] = np.log(stock_prices['Close'])
stock_prices['Open'] = np.log(stock_prices['Open'])
stock_prices['High'] = np.log(stock_prices['High'])
stock_prices['Low'] = np.log(stock_prices['Low'])
stock_prices.sample(10)

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
RowId,,,,,,,,,,,
20190425_9537,2019-04-25,9537,7.996654,7.996654,7.995644,7.995644,200,1.0,NaN,False,-0.015905
20200124_4431,2020-01-24,4431,8.137396,8.149024,8.122668,8.137396,111500,1.0,NaN,False,0.046338
20190509_6754,2019-05-09,6754,7.469654,7.477604,7.434257,7.438972,6514900,1.0,NaN,False,-0.029172
20210811_3116,2021-08-11,3116,7.678789,7.683404,7.673688,7.675546,441400,1.0,NaN,False,-0.012903
20190909_6625,2019-09-09,6625,5.262690,5.273000,5.257495,5.262690,45400,1.0,NaN,False,-0.015464
20210204_7238,2021-02-04,7238,5.147494,5.147494,5.087596,5.129899,6087100,1.0,NaN,False,0.005882
20190515_6118,2019-05-15,6118,6.806829,6.806829,6.703188,6.726233,517600,1.0,NaN,False,-0.019036
20190705_8151,2019-07-05,8151,6.903747,6.920672,6.903747,6.920672,91100,1.0,NaN,False,-0.002991
20180905_2124,2018-09-05,2124,7.824046,7.825645,7.796058,7.801391,126300,1.0,NaN,False,-0.005604


In [9]:
#Это бесполезные операции, сначала генерирурю список дней и акций, а потом джойном удаляю строки, для которых нет данных в stock_prices
#Просто потренировался, можно сразу брать индексы из stock_prices
date_range = pd.date_range(stock_prices.Date.min(), stock_prices.Date.max(), freq='D')
dates = np.array(date_range)
stocks = np.array(stock_list_incl.index)
multi_index = pd.MultiIndex.from_product([dates, stocks], names=['Date','SecuritiesCode'])
final_full = multi_index.to_frame(index = False)
final1 = final_full.merge(stock_prices.iloc[:,:], how='inner', on=['Date','SecuritiesCode'])
final1['AdjustmentFactor'] = final1.sort_values(by='Date').groupby('SecuritiesCode')['AdjustmentFactor'].shift(1)
final1['AdjustmentFactor'] = final1.sort_values(by='Date').groupby('SecuritiesCode')['AdjustmentFactor'].bfill()
final1['DoW'] = [i.weekday() for i in final1['Date']]
one_hot = pd.get_dummies(final1['DoW'])
final1 = final1.drop('DoW',axis = 1)
final1 = final1.join(one_hot)
final1 = final1.rename(columns={0: 'Mo',1: 'Tu',2: 'We',3: 'Th',4: 'Fr'})
final1[(final1.SecuritiesCode == 2590) & (final1.Date < '2017-02-01')]


,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Mo,Tu,We,Th,Fr
214,2017-01-04,2590,8.716044,8.732305,8.714403,8.725832,77400,1.0,NaN,False,-0.003200,0,0,1,0,0
2079,2017-01-05,2590,8.727454,8.743532,8.724207,8.740337,74500,1.0,NaN,False,-0.012841,0,0,0,1,0
3944,2017-01-06,2590,8.743532,8.746716,8.727454,8.737132,74500,1.0,NaN,False,-0.004878,0,0,0,0,1
5809,2017-01-10,2590,8.733916,8.733916,8.719317,8.724207,90100,1.0,NaN,False,-0.003268,0,1,0,0,0
7674,2017-01-11,2590,8.722580,8.727454,8.717682,8.719317,62200,1.0,NaN,False,-0.001639,0,0,1,0,0
9539,2017-01-12,2590,8.717682,8.720950,8.709465,8.716044,79700,1.0,NaN,False,-0.004926,0,0,0,1,0
11404,2017-01-13,2590,8.707814,8.719317,8.707814,8.714403,66100,1.0,NaN,False,-0.019802,0,0,0,0,1
13269,2017-01-16,2590,8.709465,8.720950,8.709465,8.709465,125800,1.0,30.0,False,0.005051,1,0,0,0,0
15134,2017-01-17,2590,8.706159,8.707814,8.689464,8.689464,243900,1.0,NaN,False,0.005025,0,1,0,0,0
16999,2017-01-18,2590,8.682708,8.697847,8.675905,8.694502,197000,1.0,NaN,False,-0.003333,0,0,1,0,0


In [10]:
#Магия
e = final1[final1.AdjustmentFactor != 1].set_index('Date').groupby(
    ['SecuritiesCode'])['AdjustmentFactor'].expanding(1).apply(np.prod)
e.name = 'CumFactor'
final2 = final1.merge(e, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final2['CumFactor'] = final2.sort_values(by='Date').groupby(['SecuritiesCode'])['CumFactor'].fillna(method='ffill')
final2['CumFactor'] = final2['CumFactor'].fillna(1)
final2['Open'] = final2['Open'] / final2['CumFactor']
final2['High'] = final2['High'] / final2['CumFactor']
final2['Low'] = final2['Low'] / final2['CumFactor']
final2['Close'] = final2['Close'] / final2['CumFactor']
final2['Volume'] = final2['Volume'] * final2['CumFactor']
final2['ExpectedDividend'] = (final2['ExpectedDividend'] / final2['CumFactor'])
final2['DividendRatio'] = final2['ExpectedDividend'] / final2['Close']
final2['DividendRatio'] = final2['DividendRatio'].fillna(0)
#Вопрос: в чистом виде модель всегда будет советовать вставать в шорт перед экс-дивидендой датой, в реальной жизни это не имеет смысла
#так как либо это запрещено, либо с шортящих вычтут сумму дивидендов, но формула оценки модели этого не учитывает
#можно либо пользоваться этой дырой, либо из target вычитать DividendRatio, то есть если цена упадет больше чем на сумму дивидендов
#то мы окажемся в плюсе
final2[(final2.SecuritiesCode==3421) & (final2.Date < '2017-02-01')]

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Mo,Tu,We,Th,Fr,CumFactor,DividendRatio
390,2017-01-04,3421,7.207119,7.217443,7.203406,7.215975,30700.0,1.0,NaN,False,0.002935,0,0,1,0,0,1.0,0.000000
2255,2017-01-05,3421,7.215240,7.220374,7.207860,7.217443,21900.0,1.0,NaN,False,0.003658,0,0,0,1,0,1.0,0.000000
4120,2017-01-06,3421,7.217443,7.222566,7.208600,7.220374,20300.0,1.0,NaN,False,0.002915,0,0,0,0,1,1.0,0.000000
5985,2017-01-10,3421,7.225481,7.226209,7.216709,7.224025,30000.0,1.0,NaN,False,-0.001453,0,1,0,0,0,1.0,0.000000
7850,2017-01-11,3421,7.226936,7.228388,7.223296,7.226936,20300.0,1.0,NaN,False,0.000728,0,0,1,0,0,1.0,0.000000
9715,2017-01-12,3421,7.224025,7.226209,7.213032,7.225481,22500.0,1.0,NaN,False,-0.006545,0,0,0,1,0,1.0,0.000000
11580,2017-01-13,3421,7.224025,7.226209,7.218910,7.226209,17100.0,1.0,NaN,False,-0.008785,0,0,0,0,1,1.0,0.000000
13445,2017-01-16,3421,7.226209,7.226209,7.218910,7.219642,19100.0,1.0,NaN,False,-0.004431,1,0,0,0,0,1.0,0.000000
15310,2017-01-17,3421,7.218177,7.218177,7.207860,7.210818,19000.0,1.0,NaN,False,0.008160,0,1,0,0,0,1.0,0.000000
17175,2017-01-18,3421,7.205635,7.208600,7.199678,7.206377,22900.0,1.0,NaN,False,-0.001472,0,0,1,0,0,1.0,0.000000


In [11]:
#Скользящее среднее за 50 дней
sma50 = final2.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window=50,min_periods=1).mean()
sma50.name = 'SMA50'
final3 = final2.merge(sma50, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
#Скользящее среднее за 10 дней
sma10 = final3.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window=10,min_periods=1).mean()
sma10.name = 'SMA10'
final4 = final3.merge(sma10, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
#Относительная разница цен закрытия и скользящих средних
final4['SMA50_Close_diff'] = (final4.Close-final4.SMA50) / final4.Close
final4['SMA10_Close_diff'] = (final4.Close-final4.SMA10) / final4.Close
final4

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,...,Tu,We,Th,Fr,CumFactor,DividendRatio,SMA50,SMA10,SMA50_Close_diff,SMA10_Close_diff
0,2017-01-04,1301,7.913521,7.921173,7.912057,7.916443,31400.0,1.0,NaN,False,...,0,1,0,0,1.0,0.0,7.916443,7.916443,0.000000,0.000000
1,2017-01-04,1332,6.342121,6.356108,6.333280,6.347389,2798500.0,1.0,NaN,False,...,0,1,0,0,1.0,0.0,6.347389,6.347389,0.000000,0.000000
2,2017-01-04,1333,8.055158,8.074026,8.051978,8.074026,270800.0,1.0,NaN,False,...,0,1,0,0,1.0,0.0,8.074026,8.074026,0.000000,0.000000
3,2017-01-04,1376,7.319865,7.346010,7.319865,7.346010,11300.0,1.0,NaN,False,...,0,1,0,0,1.0,0.0,7.346010,7.346010,0.000000,0.000000
4,2017-01-04,1377,8.092545,8.116716,8.092545,8.110728,150800.0,1.0,NaN,False,...,0,1,0,0,1.0,0.0,8.110728,8.110728,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336526,2021-12-07,9990,6.265301,6.282267,6.261492,6.282267,57800.0,1.0,NaN,False,...,1,0,0,0,1.0,0.0,6.330718,6.274723,-0.007712,0.001201
2336527,2021-12-07,9991,6.678342,6.692084,6.674561,6.690842,48500.0,1.0,NaN,False,...,1,0,0,0,1.0,0.0,6.737089,6.670942,-0.006912,0.002974
2336528,2021-12-07,9993,7.402452,7.402452,7.390181,7.390181,6600.0,1.0,NaN,False,...,1,0,0,0,1.0,0.0,7.444035,7.419919,-0.007287,-0.004024
2336529,2021-12-07,9994,7.798523,7.799753,7.792762,7.799753,5200.0,1.0,NaN,False,...,1,0,0,0,1.0,0.0,7.789465,7.775556,0.001319,0.003102


In [12]:
#Доля объемов на акцию от общедневного объема торговли
final4['USDVol'] = final4['Volume'] * final4['Close']
temp1 = final4.groupby('Date').agg('sum')['USDVol']
final4 = final4.rename(columns={'USDVol': 'ind_volume'})
final5 = final4.merge(temp1, how='left', left_on='Date', right_index=True)
final5 = final5.rename(columns={'USDVol': 'tot_volume'})
final5['Volume_share'] = final5['ind_volume'] / final5['tot_volume']
final5

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,...,Fr,CumFactor,DividendRatio,SMA50,SMA10,SMA50_Close_diff,SMA10_Close_diff,ind_volume,tot_volume,Volume_share
0,2017-01-04,1301,7.913521,7.921173,7.912057,7.916443,31400.0,1.0,NaN,False,...,0,1.0,0.0,7.916443,7.916443,0.000000,0.000000,2.485763e+05,1.504302e+10,0.000017
1,2017-01-04,1332,6.342121,6.356108,6.333280,6.347389,2798500.0,1.0,NaN,False,...,0,1.0,0.0,6.347389,6.347389,0.000000,0.000000,1.776317e+07,1.504302e+10,0.001181
2,2017-01-04,1333,8.055158,8.074026,8.051978,8.074026,270800.0,1.0,NaN,False,...,0,1.0,0.0,8.074026,8.074026,0.000000,0.000000,2.186446e+06,1.504302e+10,0.000145
3,2017-01-04,1376,7.319865,7.346010,7.319865,7.346010,11300.0,1.0,NaN,False,...,0,1.0,0.0,7.346010,7.346010,0.000000,0.000000,8.300992e+04,1.504302e+10,0.000006
4,2017-01-04,1377,8.092545,8.116716,8.092545,8.110728,150800.0,1.0,NaN,False,...,0,1.0,0.0,8.110728,8.110728,0.000000,0.000000,1.223098e+06,1.504302e+10,0.000081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336526,2021-12-07,9990,6.265301,6.282267,6.261492,6.282267,57800.0,1.0,NaN,False,...,0,1.0,0.0,6.330718,6.274723,-0.007712,0.001201,3.631150e+05,9.349328e+09,0.000039
2336527,2021-12-07,9991,6.678342,6.692084,6.674561,6.690842,48500.0,1.0,NaN,False,...,0,1.0,0.0,6.737089,6.670942,-0.006912,0.002974,3.245059e+05,9.349328e+09,0.000035
2336528,2021-12-07,9993,7.402452,7.402452,7.390181,7.390181,6600.0,1.0,NaN,False,...,0,1.0,0.0,7.444035,7.419919,-0.007287,-0.004024,4.877520e+04,9.349328e+09,0.000005
2336529,2021-12-07,9994,7.798523,7.799753,7.792762,7.799753,5200.0,1.0,NaN,False,...,0,1.0,0.0,7.789465,7.775556,0.001319,0.003102,4.055872e+04,9.349328e+09,0.000004


In [13]:
#Полосы Боллинджера:
#средняя линия уже есть, это SMA10
#верхняя линия: средняя линия + (10 * стандартное отклонение цен закрытия за 10 дней)
#нижняя линия: средняя линия - (10 * стандартное отклонение цен закрытия за 10 дней)
sma10stdev = final5.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window=10,min_periods=1).std()
sma10stdev.name = 'SD10'
final6 = final5.merge(sma10stdev, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final6['Upper_line'] = final6['SMA10'] + final6['SD10']
final6['Lower_line'] = final6['SMA10'] - final6['SD10']
#относительная разница между ценой закрытия и нижней/верхней линиями
final6['UL_Close-diff'] = (final6.Close-final6.Upper_line) / final6.Close
final6['LL_Close-diff'] = (final6.Close-final6.Lower_line) / final6.Close
final6

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,...,SMA50_Close_diff,SMA10_Close_diff,ind_volume,tot_volume,Volume_share,SD10,Upper_line,Lower_line,UL_Close-diff,LL_Close-diff
0,2017-01-04,1301,7.913521,7.921173,7.912057,7.916443,31400.0,1.0,NaN,False,...,0.000000,0.000000,2.485763e+05,1.504302e+10,0.000017,NaN,NaN,NaN,NaN,NaN
1,2017-01-04,1332,6.342121,6.356108,6.333280,6.347389,2798500.0,1.0,NaN,False,...,0.000000,0.000000,1.776317e+07,1.504302e+10,0.001181,NaN,NaN,NaN,NaN,NaN
2,2017-01-04,1333,8.055158,8.074026,8.051978,8.074026,270800.0,1.0,NaN,False,...,0.000000,0.000000,2.186446e+06,1.504302e+10,0.000145,NaN,NaN,NaN,NaN,NaN
3,2017-01-04,1376,7.319865,7.346010,7.319865,7.346010,11300.0,1.0,NaN,False,...,0.000000,0.000000,8.300992e+04,1.504302e+10,0.000006,NaN,NaN,NaN,NaN,NaN
4,2017-01-04,1377,8.092545,8.116716,8.092545,8.110728,150800.0,1.0,NaN,False,...,0.000000,0.000000,1.223098e+06,1.504302e+10,0.000081,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336526,2021-12-07,9990,6.265301,6.282267,6.261492,6.282267,57800.0,1.0,NaN,False,...,-0.007712,0.001201,3.631150e+05,9.349328e+09,0.000039,0.030360,6.305083,6.244362,-0.003632,0.006034
2336527,2021-12-07,9991,6.678342,6.692084,6.674561,6.690842,48500.0,1.0,NaN,False,...,-0.006912,0.002974,3.245059e+05,9.349328e+09,0.000035,0.021320,6.692262,6.649622,-0.000212,0.006161
2336528,2021-12-07,9993,7.402452,7.402452,7.390181,7.390181,6600.0,1.0,NaN,False,...,-0.007287,-0.004024,4.877520e+04,9.349328e+09,0.000005,0.020048,7.439967,7.399871,-0.006737,-0.001311
2336529,2021-12-07,9994,7.798523,7.799753,7.792762,7.799753,5200.0,1.0,NaN,False,...,0.001319,0.003102,4.055872e+04,9.349328e+09,0.000004,0.014737,7.790293,7.760819,0.001213,0.004992


In [14]:
#RSI
#а) среднее значение последних 14 растущих дневных свечей
#б) среднее значение последних 14 падающих дневных свечей
#в) считаем RS: делим (а) на (б)
#г) считаем RSI: 100-(100/(1+RS))

last14up = final6[final6["Close"] > final6["Open"]].set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window=14, min_periods=1).mean()
last14up.name = 'SMA14UP'
final7 = final6.merge(last14up, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final7['SMA14UP'] = final7.sort_values(by='Date').groupby(['SecuritiesCode'])['SMA14UP'].fillna(method='ffill')
last14down = final7[final7["Close"] < final7["Open"]].set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window=14, min_periods=1).mean()
last14down.name = 'SMA14DOWN'
final8 = final7.merge(last14down, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final8['SMA14DOWN'] = final8.sort_values(by='Date').groupby(['SecuritiesCode'])['SMA14DOWN'].fillna(method='ffill')
final8['RSI'] = (100-(100/(1+(final8.SMA14UP / final8.SMA14DOWN))))/100
final8
final8[final8.SecuritiesCode == 9993].tail(20)
#last14up[9993].tail(20)


,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,...,tot_volume,Volume_share,SD10,Upper_line,Lower_line,UL_Close-diff,LL_Close-diff,SMA14UP,SMA14DOWN,RSI
2298528,2021-11-09,9993,7.447168,7.450661,7.446001,7.448334,2500.0,1.0,NaN,False,...,8.667187e+09,2.148429e-06,0.003887,7.451864,7.444091,-0.000474,0.000570,7.457097,7.444865,0.500410
2300528,2021-11-10,9993,7.454720,7.454720,7.449498,7.449498,2400.0,1.0,NaN,False,...,8.308035e+09,2.151989e-06,0.003907,7.451943,7.444129,-0.000328,0.000721,7.457097,7.444865,0.500410
2302528,2021-11-11,9993,7.447168,7.447168,7.446585,7.446585,300.0,1.0,NaN,False,...,8.346653e+09,2.676493e-07,0.002097,7.450953,7.446759,-0.000587,-0.000023,7.457097,7.444948,0.500408
2304528,2021-11-12,9993,7.452402,7.452402,7.447751,7.450661,1000.0,1.0,NaN,False,...,9.286897e+09,8.022767e-07,0.001676,7.450300,7.446948,0.000048,0.000498,7.457097,7.445365,0.500394
2306528,2021-11-15,9993,7.448334,7.450661,7.446585,7.447751,1100.0,1.0,NaN,False,...,8.314639e+09,9.853135e-07,0.001612,7.450003,7.446779,-0.000302,0.000130,7.457097,7.445490,0.500389
2308528,2021-11-16,9993,7.454141,7.454141,7.448916,7.452982,1500.0,1.0,NaN,False,...,8.503564e+09,1.314681e-06,0.002030,7.451061,7.447000,0.000258,0.000803,7.457097,7.445863,0.500377
2310528,2021-11-17,9993,7.450661,7.457609,7.450661,7.450661,2100.0,1.0,NaN,False,...,8.368816e+09,1.869606e-06,0.001777,7.451273,7.447720,-0.000082,0.000395,7.457097,7.445863,0.500377
2312528,2021-11-18,9993,7.450661,7.453562,7.450661,7.451242,1200.0,1.0,NaN,False,...,8.696028e+09,1.028227e-06,0.001861,7.451531,7.447810,-0.000039,0.000460,7.455295,7.445863,0.500316
2314528,2021-11-19,9993,7.455877,7.458763,7.451822,7.452982,1200.0,1.0,NaN,False,...,8.506858e+09,1.051338e-06,0.002105,7.452183,7.447972,0.000107,0.000672,7.455295,7.446906,0.500281
2316528,2021-11-22,9993,7.454720,7.465083,7.454141,7.465083,5100.0,1.0,NaN,False,...,7.504348e+09,5.073315e-06,0.005191,7.456769,7.446387,0.001114,0.002504,7.455050,7.446906,0.500273


In [15]:
#Стохастический
#а) k = (цена закрытия - мин. цена дня)/(макс. цена дня - мин. цена дня)
#б) d = усредненная k за 3 дн
#теоретически возможно, что в знаменателе будет 0 три дня подряд и среднее не посчитается,
#на практике за 5 лет это не произошло ни разу, поэтому обрабатывать такое исключение не буду
final8["k"] = (final8["Close"]-final8["Low"])/((final8["High"]-final8["Low"]))
stohastic = final8.set_index('Date').groupby(['SecuritiesCode'])['k'].rolling(window=3, min_periods=1).mean()
stohastic.name = 'Stochastic'
final9 = final8.merge(stohastic, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final9.fillna(0, inplace=True) #, subset=['k'])

In [16]:
#Ебанина
#Индикатор схождения-расхождения скользящих средних (MACD)
# а) Экспоненциальная скользящая средняя за 12 дней: EMA = Close * 2/(n+1) + EMA(prev) * (1–2/(n+1))
# б) Экспоненциальная скользящая средняя за 26 дней: EMA = Close * 2/(n+1) + EMA(prev) * (1–2/(n+1))
# в) MACD = ЕМА_12 − EMA_26
# г) Экспоненциальная скользящая средняя за 9 дней: EMA = MACD * 2/(n+1) + MACD(prev) * (1–2/(n+1))
# е) разница между EMA9 и MACD

ema12 = final9.set_index('Date').groupby(['SecuritiesCode'])['Close'].ewm(span=12, adjust=False).mean()
ema12.name = 'EMA12'
ema26 = final9.set_index('Date').groupby(['SecuritiesCode'])['Close'].ewm(span=26, adjust=False).mean()
ema26.name = 'EMA26'
final10 = final9.merge(ema12, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final11 = final10.merge(ema26, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final11['MACD'] = final11['EMA12'] - final11['EMA26']
ema9 = final11.set_index('Date').groupby(['SecuritiesCode'])['MACD'].ewm(span=9, adjust=False).mean()
ema9.name = 'EMA9'
final12 = final11.merge(ema9, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final12['EMA_diff'] = (final12['EMA9'] - final12['MACD']) / final12['MACD']

In [17]:
final12['BeforeDiv'] = final12.sort_values(by='Date').groupby(
    ['SecuritiesCode'])['ExpectedDividend'].fillna(method='bfill', limit=10)
final12['BeforeDiv'] = ((final12['BeforeDiv'] > 0) & (final12['BeforeDiv'] != final12['ExpectedDividend'])).astype(int)
final12['AfterDiv'] = final12.sort_values(by='Date').groupby(
    ['SecuritiesCode'])['ExpectedDividend'].fillna(method='ffill', limit=10)
final12['AfterDiv'] = ((final12['AfterDiv'] > 0) & (final12['AfterDiv'] != final12['ExpectedDividend'])).astype(int)


Addtional features

In [18]:
#functions for the feature generation
#rolling features
def rolling_feature(df, window=50, op='mean'):
#input: a stock_prices-like df, the window param for rolling and type of aggregation
#output: df with a merged column of the needed rolling feature
    if op == 'mean':
        sma = df.set_index('Date').groupby(
            ['SecuritiesCode'])['Close'].rolling(
            window=window,min_periods=1).mean()
        sma.name = 'Sliding_mean_' + str(window)
        result = df.merge(sma, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
        return result
    elif op == 'median':
        sma = df.set_index('Date').groupby(
            ['SecuritiesCode'])['Close'].rolling(
            window=window,min_periods=1).apply(np.median, raw=True, engine='numba')
        sma.name = 'Sliding_median_' + str(window)
        result = df.merge(sma, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
        return result
    else:
        print('Invalid input')

#lag generation
def lags(df, lag=1, column='Close'):
#input: stock-price-like df with the columns Date, SecuritiesCode and the column to lag
#output: input df with the additional lagged column
    if column + '_lag_' + str(lag) not in df.columns:
        if 'Day_number' in df.columns:
            result = df
        else:
            prices_by_date = df.groupby(['Date']).agg({'SecuritiesCode' : 'count'}).reset_index()
            prices_by_date['Day_number'] = prices_by_date.index
            result = df.merge(prices_by_date[['Date', 'Day_number']], how='left', on='Date')
            del(prices_by_date)

        stock_prices_to_lag = result[['Day_number', 'SecuritiesCode', column]]
        stock_prices_to_lag['Day_number'] += lag
        stock_prices_to_lag.rename(columns={column : column + '_lag_' + str(lag)}, inplace=True)
        result = result.merge(
            stock_prices_to_lag[['Day_number', 'SecuritiesCode', column + '_lag_' + str(lag)]], 
            how='left', on=['Day_number', 'SecuritiesCode'])
        del(stock_prices_to_lag)
        return result
    else:
        return df


#candles
def candle(row):
#function to use in .apply
#row must contain 3 lags for open and close
#source: https://www.investopedia.com/trading/candlestick-charting-what-is-it/
    seq = ''
    if row['Close_lag_3'] > row['Open_lag_3']:
        seq += 'g'
    elif row['Close_lag_3'] < row['Open_lag_3']:
        seq += 'r'
        
    if row['Close_lag_2'] > row['Open_lag_2']:
        seq += 'g'
    elif row['Close_lag_2'] < row['Open_lag_2']:
        seq += 'r'
        
    if row['Close_lag_1'] > row['Open_lag_1']:
        if abs(row['Close_lag_1'] - row['Open_lag_1']) < abs(row['Close_lag_2'] - row['Open_lag_2']):
            seq += 'g'
        else: 
            seq += 'G'
    elif row['Close_lag_1'] < row['Open_lag_1']:
        if abs(row['Close_lag_1'] - row['Open_lag_1']) < abs(row['Close_lag_2'] - row['Open_lag_2']):
            seq += 'r'
        else:
            seq += 'R'
    
    if row['Close'] > row['Open']:
        if (row['Close_lag_1'] < row['Close'] < row['Open_lag_1'] and
            row['Close_lag_1'] < row['Open'] < row['Open_lag_1']):
            seq += 'g_e'
        else:
            seq += 'g'
    else:
        if (row['Open_lag_1'] < row['Close'] < row['Close_lag_1'] and
            row['Open_lag_1'] < row['Open'] < row['Close_lag_1']):
            seq += 'r_e'
        else:
            seq += 'r'
            
    #outcomes
    if seq == 'ggGr_e':#Bearish Harami
        return -1
    elif seq == 'gggr': #Bearish Engulfing Pattern
        return -1
    elif seq == 'ggrr': #Bearish Evening Star
        return -1
    elif seq == 'rrrg': #Bullish Engulfing Pattern
        return 1
    elif seq == 'rrRg_e': #Bullish Harami
        return 1
    else:
        return 0      

In [19]:
#weighted difference between yesterday's close and today's open
final13 = lags(final12, lag = 1, column='Close')
def gap(row):
    if row['Close_lag_1'] != 0:
        return (row['Open'] - row['Close_lag_1']) / row['Close_lag_1']
    return 0
final13['Gap'] = final13.apply(gap, axis=1)

/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_prices_to_lag['Day_number'] += lag
/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_prices_to_lag.rename(columns={column : column + '_lag_' + str(lag)}, inplace=True)


In [20]:
#candles
final14 = final13
for i in range(1, 4):
    final14 = lags(final14, lag=i, column='Close')
    final14 = lags(final14, lag=i, column='Open')
final14['Candle'] = final14.apply(candle, axis=1)

/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_prices_to_lag['Day_number'] += lag
/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_prices_to_lag.rename(columns={column : column + '_lag_' + str(lag)}, inplace=True)
/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [21]:
#target lags
final15 = final14
for i in range(2, 5):
    final15 = lags(final15, lag=i, column='Target')
final15

/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_prices_to_lag['Day_number'] += lag
/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_prices_to_lag.rename(columns={column : column + '_lag_' + str(lag)}, inplace=True)
/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_18949/834123432.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,...,Gap,Open_lag_1,Close_lag_2,Open_lag_2,Close_lag_3,Open_lag_3,Candle,Target_lag_2,Target_lag_3,Target_lag_4
0,2017-01-04,1301,7.913521,7.921173,7.912057,7.916443,31400.0,1.0,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1,2017-01-04,1332,6.342121,6.356108,6.333280,6.347389,2798500.0,1.0,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,2017-01-04,1333,8.055158,8.074026,8.051978,8.074026,270800.0,1.0,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,2017-01-04,1376,7.319865,7.346010,7.319865,7.346010,11300.0,1.0,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
4,2017-01-04,1377,8.092545,8.116716,8.092545,8.110728,150800.0,1.0,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336526,2021-12-07,9990,6.265301,6.282267,6.261492,6.282267,57800.0,1.0,0.0,False,...,0.002762,6.269096,6.269096,6.242223,6.228511,6.249975,0,0.034816,-0.020833,0.041420
2336527,2021-12-07,9991,6.678342,6.692084,6.674561,6.690842,48500.0,1.0,0.0,False,...,0.001899,6.679599,6.677083,6.661855,6.656727,6.655440,0,0.025478,-0.011335,0.020566
2336528,2021-12-07,9993,7.402452,7.402452,7.390181,7.390181,6600.0,1.0,0.0,False,...,0.001076,7.405496,7.405496,7.432484,7.408531,7.421776,0,-0.004302,-0.010942,-0.003030
2336529,2021-12-07,9994,7.798523,7.799753,7.792762,7.799753,5200.0,1.0,0.0,False,...,0.001005,7.780721,7.778630,7.778211,7.767264,7.757906,0,0.009098,0.012139,0.011431


In [22]:
len(final15[final15['Date']=='2021-12-06'])

2000

In [24]:
#pickling the df
final15.to_parquet('./log_features_220526.parquet.gzip',
              compression='gzip')

In [ ]:
final16 = final15.drop(['Date','SecuritiesCode','Open','High','Low','Close','Volume',
                        'AdjustmentFactor','ExpectedDividend','SupervisionFlag','CumFactor',
                       'SMA50','SMA10','k','EMA12','EMA26','SMA14UP','SMA14DOWN','Lower_line','Upper_line',
                       'SD10','MACD','EMA9','tot_volume','ind_volume'], axis=1)
msno.matrix(final13)

In [ ]:
data1 =  final13[['Target','DividendRatio']]
sns.scatterplot(data=data1, x="Target", y="DividendRatio")
plt.show()

In [ ]:
data1 =  final13[['Target','EMA_diff']]
sns.scatterplot(data=data1, x="Target", y="EMA_diff")
plt.show()

In [ ]:
final13.groupby('')

In [ ]:
options

In [ ]:
options1 = options.copy()
#Все опционы на Nikkei Stock Average (Nikkei 225) 
years = {'6':2021,'7':2022,'8':2023,'9':2024,'0':2025,'1':2026,'2':2027,'3':2028,'4':2029,'5':2030,'6':2031,'7':2032}
months = {'01':1,'02':2,'03':3,'04':4,'05':5,'06':6,'07':7,'08':8,'09':9,'10':10,'11':11,'12':12}
types = {'1':'Put featues','2':'Call features','3':'Put spot','4':'Call spot','5':'Put featues','6':'Features','7':'Call features','8':'Put spot','9':'Call spot'}

options1['OptionsCode'] = options1['OptionsCode'].apply(str)
options1['Contract_Type'] = [types[i[1]] for i in options1['OptionsCode']]
options1['Contract_Year'] = [years[i[2]] for i in options1['OptionsCode']]
options1['Ex_price'] = [i[5:7] for i in options1['OptionsCode']]

#option1 = options[options['Putcall'] == 1].groupby(['Date','OptionsCode'[:5]]).agg('sum')['OpenInterest']
#unique_numbers = set(options['Index'])
#unique_numbers



In [ ]:
put = options[options['Putcall'] == 1].groupby('Date').agg('sum')['OpenInterest'].reset_index()
put['PrevOpenInterest'] = put['OpenInterest'].shift(1)
put['Diff'] = (put['OpenInterest'] - put['PrevOpenInterest']) / put['PrevOpenInterest']
put

In [ ]:
call = options[options['Putcall'] == 2].groupby('Date').agg('sum')['OpenInterest'].reset_index()
call['PrevOpenInterest'] = call['OpenInterest'].shift(1)
call['Diff'] = (call['OpenInterest'] - call['PrevOpenInterest']) / call['PrevOpenInterest']
call